In [1]:
!pip install powerlaw
import pandas as pd
import glob
import numpy as np
import pickle
import networkx as nx
import itertools
import matplotlib.pyplot as plt
from collections import Counter
import powerlaw
import math
import operator

In [2]:
unique_subreddits_path =r"cleaned_data\unique_subreddits"
with open(unique_subreddits_path, 'rb') as file:
    unique_subreddits = pickle.load(file)

with open("cleaned_data\submissions_cleanead", 'rb') as file:
    submissions = pickle.load(file)

with open("cleaned_data\comments_cleanead", 'rb') as file:
    comments = pickle.load(file)

In [3]:
print(submissions)
print(comments)

            id        author  created_utc   subreddit subreddit_id  \
0        648oo         DITUS   1199145615  reddit.com         t5_6   
1        648op        rmuser   1199145634     science      t5_mouw   
3        648or         zorno   1199145709  reddit.com         t5_6   
4        648os        rmuser   1199145735    politics     t5_2cneq   
5        648ot         zorno   1199145735  reddit.com         t5_6   
...        ...           ...          ...         ...          ...   
2519847  7amgd     hunkydude   1225497545      Health     t5_2qh9z   
2519848  7amge  violetplanet   1225497549    BlackOps     t5_2qhwv   
2519850  7amgg    justblogit   1225497564  reddit.com         t5_6   
2519851  7amgh  guillebravo6   1225497573     offbeat     t5_2qh11   
2519852  7amgi     qazamisan   1225497591  reddit.com         t5_6   

         num_comments  score  over_18  
0                   1      0    False  
1                   5     20    False  
3                   1      3    False  

In [29]:
comment_collumns_to_use = ['id','author','link_id','parent_id']
comments_filtered = comments[comment_collumns_to_use]
print(comments_filtered)

submission_collumns_to_use = ['id','author', 'num_comments']
submissions_filtered = submissions[submission_collumns_to_use]
print(submissions_filtered)



              id               author   link_id   parent_id
0        c02s9s6                Haven  t3_648oh  t1_c02s9rv
2        c02s9s8             lilmiss2  t3_648oh  t1_c02s9rv
6        c02s9sc  EverybodysAnAsshole  t3_648et  t1_c02s976
7        c02s9sd             generalk  t3_647yd  t1_c02s8md
8        c02s9se            seeker135  t3_6483n    t3_6483n
...          ...                  ...       ...         ...
7242866  c064gsy         rhythmicidea  t3_7amdg  t1_c064gqo
7242867  c064gsz       blackwhitegrey  t3_7ajij    t3_7ajij
7242868  c064gt0           Drewster58  t3_7amg7    t3_7amg7
7242869  c064gt1               eerock  t3_7akoc    t3_7akoc
7242870  c064gt2               mycall  t3_7ajjm    t3_7ajjm

[4873689 rows x 4 columns]
            id        author  num_comments
0        648oo         DITUS             1
1        648op        rmuser             5
3        648or         zorno             1
4        648os        rmuser             2
5        648ot         zorno         

In [27]:
df = comments_filtered.reset_index()  # make sure indexes pair with number of rows

for index, row in df.iterrows():
    link_id = row['link_id'][3:]
    df.iat[index, df.columns.get_loc('link_id')] = link_id
    parent_id = row['parent_id'][3:]
    df.iat[index, df.columns.get_loc('parent_id')] = parent_id 

print(df)

           index       id               author link_id parent_id
0              0  c02s9s6                Haven   648oh   c02s9rv
1              2  c02s9s8             lilmiss2   648oh   c02s9rv
2              6  c02s9sc  EverybodysAnAsshole   648et   c02s976
3              7  c02s9sd             generalk   647yd   c02s8md
4              8  c02s9se            seeker135   6483n     6483n
...          ...      ...                  ...     ...       ...
4873684  7242866  c064gsy         rhythmicidea   7amdg   c064gqo
4873685  7242867  c064gsz       blackwhitegrey   7ajij     7ajij
4873686  7242868  c064gt0           Drewster58   7amg7     7amg7
4873687  7242869  c064gt1               eerock   7akoc     7akoc
4873688  7242870  c064gt2               mycall   7ajjm     7ajjm

[4873689 rows x 5 columns]


In [30]:
#Remove submissiong that have 0 number of comments
submissions_with_comments_mask = submissions_filtered['num_comments'] > 0
submissions_with_comments = submissions_filtered[submissions_with_comments_mask]
print(submissions_with_comments)

            id         author  num_comments
0        648oo          DITUS             1
1        648op         rmuser             5
3        648or          zorno             1
4        648os         rmuser             2
6        648ou  anteeknetshop             1
...        ...            ...           ...
2519831  7amfx      oddietaco             4
2519841  7amg7     Drewster58             1
2519843  7amg9      clhibbard             1
2519850  7amgg     justblogit             1
2519851  7amgh   guillebravo6             5

[797907 rows x 3 columns]


In [32]:
comments = df
submissions = submissions_with_comments[['id','author']]
print(comments)
print(submissions)

           index       id               author link_id parent_id
0              0  c02s9s6                Haven   648oh   c02s9rv
1              2  c02s9s8             lilmiss2   648oh   c02s9rv
2              6  c02s9sc  EverybodysAnAsshole   648et   c02s976
3              7  c02s9sd             generalk   647yd   c02s8md
4              8  c02s9se            seeker135   6483n     6483n
...          ...      ...                  ...     ...       ...
4873684  7242866  c064gsy         rhythmicidea   7amdg   c064gqo
4873685  7242867  c064gsz       blackwhitegrey   7ajij     7ajij
4873686  7242868  c064gt0           Drewster58   7amg7     7amg7
4873687  7242869  c064gt1               eerock   7akoc     7akoc
4873688  7242870  c064gt2               mycall   7ajjm     7ajjm

[4873689 rows x 5 columns]
            id         author
0        648oo          DITUS
1        648op         rmuser
3        648or          zorno
4        648os         rmuser
6        648ou  anteeknetshop
...        .

In [42]:
mask = comments['link_id'].isin(submissions['id'])
comments_filtered = comments[mask]
print(comments_filtered)

           index       id           author link_id parent_id
25            34  c02s9t4  LiberalDemocrat   648os     648os
55            72  c02s9u6        DiscoWolf   648os     648os
63            83  c02s9uh         Flemlord   648p5     648p5
69            89  c02s9un  LiberalDemocrat   648oy     648oy
72            94  c02s9us  LiberalDemocrat   648p5     648p5
...          ...      ...              ...     ...       ...
4873684  7242866  c064gsy     rhythmicidea   7amdg   c064gqo
4873685  7242867  c064gsz   blackwhitegrey   7ajij     7ajij
4873686  7242868  c064gt0       Drewster58   7amg7     7amg7
4873687  7242869  c064gt1           eerock   7akoc     7akoc
4873688  7242870  c064gt2           mycall   7ajjm     7ajjm

[3781518 rows x 5 columns]


In [43]:
submissions_unique_users = submissions['author'].unique()
comments_unique_users = comments_filtered['author'].unique()

unique_users = list(set(np.concatenate([submissions_unique_users, comments_unique_users])))
print('Number of unique users : ', len(unique_users))

Number of unique users :  152730


In [44]:
#Save unique users for graph nodes
unique_users_path =r"cleaned_data\unique_users"
with open(unique_users_path, 'wb') as file:
    pickle.dump(unique_users, file)
    
#Save submissions for UserNet
user_net_submissions =r"cleaned_data\user_net_submissions"
with open(user_net_submissions, 'wb') as file:
    pickle.dump(submissions, file)
    
#Save comments for UserNet
user_net_comments =r"cleaned_data\user_net_comments"
with open(user_net_comments, 'wb') as file:
    pickle.dump(comments_filtered, file)